In [40]:
import warnings
import flwr as fl
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.linear_model import SGDOneClassSVM
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from typing import Tuple, Union, List
import numpy as np
import pandas as pd
from random import randint
from sklearn import metrics


In [35]:
XY = Tuple[np.ndarray, np.ndarray]
Dataset = Tuple[XY, XY]
ModelParams = Union[XY, Tuple[np.ndarray]]
XYList = List[XY]
Model = SGDOneClassSVM

In [36]:
def get_model_parameters(model: Model) -> ModelParams:
    """Returns the paramters of a sklearn LogisticRegression model."""
    coef_ = model.coef_
    # intercept_= model.intercept_
    params = (coef_, )
    # if model.fit_intercept:
    #     params = (model.coef_, model.intercept_)
    # else:
    #     params = (model.coef_,)
    return params


def set_model_params(
    model: Model, params: ModelParams
) -> Model:
    """Sets the parameters of a sklean LogisticRegression model."""
    model.coef_ = params[0]
    # model.intercept_ = params[1]
    # if model.fit_intercept:
    #     model.intercept_ = params[1]
    return model


def set_initial_params(model: Model):
    """Sets initial parameters as zeros Required since model params are
    uninitialized until model.fit is called.
    But server asks for initial parameters from clients at launch. Refer
    to sklearn.linear_model.LogisticRegression documentation for more
    information.
    """
    n_classes = 1  # MNIST has 10 classes
    n_features = 784  # Number of features in dataset
    model.classes_ = np.array([i for i in range(10)])

    model.coef_ = np.zeros((n_classes, n_features))
    model.offset_ = np.zeros(1)
    # if model.fit_intercept:
    #     model.intercept_ = np.zeros((n_classes,))

In [37]:
def load_mnist() -> Dataset:
    """Loads the MNIST dataset from local csv.
    """
    mnist_df = pd.read_csv("test.csv")
    print("read data finished")
    Xy = mnist_df.to_numpy()
    X = Xy[:, :-1]  # the last column contains labels
    y = [randint(0,1) for i in range(0,len(X))]
    y = [-1 if i==0 else i for i in y]
    # First 60000 samples consist of the train set
    x_train, y_train = X[:20], y[:20]
    x_test, y_test = X[20:], y[20:]
    return (x_train, y_train), (x_test, y_test)

In [38]:
def shuffle(X: np.ndarray, y: np.ndarray) -> XY:
    """Shuffle X and y."""
    rng = np.random.default_rng()
    idx = rng.permutation(len(X))
    return X[idx], y[idx]


def partition(X: np.ndarray, y: np.ndarray, num_partitions: int) -> XYList:
    """Split X and y into a number of partitions."""
    return list(
        zip(np.array_split(X, num_partitions), np.array_split(y, num_partitions))
    )

In [39]:
if __name__ == "__main__":
    # Load MNIST dataset from https://www.openml.org/d/554
    (X_train, y_train), (X_test, y_test) = load_mnist()

    # Split train set into 10 partitions and randomly use one for training.
    partition_id = np.random.choice(10)
    (X_train, y_train) = partition(X_train, y_train, 10)[partition_id]

    # Create LogisticRegression Model
    model = SGDOneClassSVM(
        
    )

    # Setting initial parameters, akin to model.compile for keras models
    set_initial_params(model)

    # Define Flower client
    class MnistClient(fl.client.NumPyClient):
        def get_parameters(self):  # type: ignore
            return get_model_parameters(model)

        def fit(self, parameters, config):  # type: ignore
            print("start fitting")
            set_model_params(model, parameters)
            # Ignore convergence failure due to low local epochs

            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model.fit(X_train)
            print(f"Training finished for round {config['rnd']}")
            return get_model_parameters(model), len(X_train), {}

        def evaluate(self, parameters, config):  # type: ignore
            print("start evaluating")
            set_model_params(model, parameters)
            # loss = log_loss(y_test, model.score_samples(X_test))
            y_pred = model.predict(X_test)
            sco = model.score_samples(X_test)
            loss = log_loss(y_test, sco)
            accuracy = metrics.accuracy_score(y_test, y_pred)
            # accuracy = model.score(X_test, y_test)
            print(f"evaluation finished for round {config['rnd']}, accuracy is {accuracy}")
            return loss, len(X_test), {"accuracy": accuracy}

    # Start Flower client
    fl.client.start_numpy_client("192.168.1.105:8080", client=MnistClient())


DEBUG flower 2021-11-17 16:15:09,029 | connection.py:36 | ChannelConnectivity.IDLE
INFO flower 2021-11-17 16:15:09,031 | app.py:61 | Opened (insecure) gRPC connection
DEBUG flower 2021-11-17 16:15:09,031 | connection.py:36 | ChannelConnectivity.TRANSIENT_FAILURE


read data finished


DEBUG flower 2021-11-17 16:15:09,241 | connection.py:68 | Insecure gRPC channel closed


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1637162109.031000000","description":"Failed to pick subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":3009,"referenced_errors":[{"created":"@1637162109.031000000","description":"failed to connect to all addresses","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":398,"grpc_status":14}]}"
>